<a href="https://colab.research.google.com/github/Felipe-Ulloa-CA/used-car-price-analysis-berkeley/blob/main/notebooks/used_car_price_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Used Car Price Analysis
UC Berkeley – Professional Certificate in Machine Learning & AI  
Practical Application 11.1 – *What Determines the Price of a Car?*

This notebook follows the **CRISP-DM** framework:

1. Business Understanding  
2. Data Understanding  
3. Data Preparation  
4. Modeling  
5. Evaluation & Recommendations

The goal is to understand which factors drive the price of a used car and to build a predictive model that can help used car dealers optimize their inventory and pricing strategy.


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Configure basic display options
pd.set_option("display.max_columns", 100)
sns.set(style="whitegrid", context="notebook")



## 1. Business Understanding

Used car dealers need to know **what makes one used car more expensive than another**.  

**Business question**

> Which features of a used car (age, mileage, brand, condition, fuel type, etc.) are most strongly associated with its price?

**Stakeholder**

- A group of used car dealers interested in:
  - Understanding what consumers value in a used car
  - Prioritizing which cars to purchase for inventory
  - Setting more accurate and data-driven prices

We will answer this question using the dataset of used vehicles and standard regression techniques.


## 2. Data Understanding

In this section, I load the used vehicles dataset and perform an initial exploration
to understand its structure, variable types, and basic statistics before further analysis.

In [ ]:
import os
import pandas as pd

data_path = "vehicles.csv"

if not os.path.exists(data_path):
    raise FileNotFoundError(
        "The file 'vehicles.csv' was not found.\n"
        "Please download the dataset from the course platform and place it in the "
        "same directory as this notebook, or update 'data_path' accordingly."
    )

df = pd.read_csv(data_path)
df.head()

The dataset was successfully loaded. The first few rows provide an overview
of the available variables, including price, vehicle characteristics, and seller information.
This confirms that the dataset contains both numerical and categorical features,
which will require different preprocessing steps later.

In [ ]:
df.shape, df.info()

The dataset contains a large number of observations and a mix of data types.
Several variables include missing values, particularly in condition, cylinders,
and manufacturer. These issues will be addressed during the data preparation stage.

In [ ]:
df.describe().T

From this initial exploration, it is clear that price shows substantial variability
and potential skewness, while mileage and vehicle age are likely to be important
predictors. This motivates a deeper exploratory analysis of price relationships
in the next sections.

### 2.1 Target variable: price

We begin by examining the distribution of the target variable (price).
Understanding the distribution helps identify skewness, potential outliers,
and whether transformations (such as log-price) may be necessary before
modeling.


In [ ]:
plt.figure(figsize=(8,5))
plt.hist(df['price'].dropna(), bins=50)
plt.title("Distribution of Used Car Prices")
plt.xlabel("Price")
plt.ylabel("Count")
plt.show()

The histogram shows that used car prices are right-skewed, with a large
concentration of lower-priced vehicles and a long tail of higher prices.
This indicates substantial variability in prices, especially among higher-end
vehicles.


### 2.2 Relationship between price and key continuous variables

We focus on **mileage** (`odometer`) and **year**, which are likely to be strong drivers of price.

Mileage is expected to strongly influence used car prices, as higher vehicle
usage typically leads to faster depreciation.


In [ ]:
# Price vs mileage
plt.figure(figsize=(8,5))
plt.scatter(df['odometer'], df['price'], alpha=0.1)
plt.title("Price vs. Mileage (Odometer)")
plt.xlabel("Odometer")
plt.ylabel("Price")
plt.show()

The scatter plot shows a clear negative relationship between mileage and price.
Vehicles with higher odometer readings tend to sell for significantly lower prices,
confirming mileage as a key determinant of used car value.

In [ ]:
# Price vs year
plt.figure(figsize=(8,5))
plt.scatter(df['year'], df['price'], alpha=0.1)
plt.title("Price vs. Year")
plt.xlabel("Year")
plt.ylabel("Price")
plt.show()

The plot indicates a positive relationship between vehicle year and price.
Newer cars tend to sell for higher prices, reinforcing vehicle age as a major
pricing factor in the used car market.

### 2.3 Price by categorical features

In addition to continuous variables, categorical features such as manufacturer and vehicle condition can have a strong influence on used car prices. We examine price distributions across these categories using boxplots.

Different manufacturers are known to retain value differently in the used car
market. By comparing price distributions across the most common manufacturers,
we assess which brands tend to command higher resale prices.


In [ ]:
# Top manufacturers by count
top_makes = df['manufacturer'].value_counts().head(10).index

plt.figure(figsize=(10,6))
sns.boxplot(data=df[df['manufacturer'].isin(top_makes)],
            x='manufacturer', y='price')
plt.xticks(rotation=45)
plt.title("Price Distribution by Manufacturer (Top 10)")
plt.xlabel("Manufacturer")
plt.ylabel("Price")
plt.show()

The boxplot shows substantial variation in prices across manufacturers.
Certain brands consistently exhibit higher median prices, indicating stronger
brand reputation and better resale value, while others show lower price levels.


Vehicle condition reflects overall wear and maintenance quality, which should
directly affect resale value. We analyze price distributions across condition
categories to evaluate this relationship.


In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(data=df, x='condition', y='price')
plt.title("Price Distribution by Condition")
plt.xlabel("Condition")
plt.ylabel("Price")
plt.show()

The results reveal clear price differences across condition categories.
Vehicles in excellent or good condition command significantly higher prices,
whereas cars in fair or poor condition are associated with substantial discounts.


## 3. Data Preparation

We now clean the data, engineer new features, and prepare the dataset for modeling.

Steps:
1. Remove missing or unrealistic prices  
2. Create **age** from `year`  
3. Filter to rows with key information (year, odometer, manufacturer, etc.)  
4. Select a subset of relevant features  
5. Encode categorical variables using one-hot encoding  
6. Split into training and test sets


### 3.1 Data cleaning and feature engineering

Before modeling, I clean the dataset by removing invalid or missing prices,
filtering extreme outliers, and ensuring key variables are present.

I also engineer an `age` feature from `year`, since age is more directly
interpretable than year when explaining depreciation effects.

In [ ]:
# 3.1 Remove rows with missing or zero/negative price
df_clean = df.dropna(subset=['price']).copy()
df_clean = df_clean[df_clean['price'] > 0]

# Remove extreme price outliers (simple rule-of-thumb thresholds)
df_clean = df_clean[df_clean['price'].between(500, 100000)]

# 3.2 Create age feature from year
reference_year = int(df_clean['year'].max())
df_clean = df_clean[df_clean['year'].notna()]
df_clean['age'] = reference_year - df_clean['year']

# 3.3 Keep rows with non-missing key features
key_cols = ['odometer', 'manufacturer', 'condition', 'fuel',
            'transmission', 'type', 'state']
df_clean = df_clean.dropna(subset=key_cols)

# 3.4 Select modeling columns
model_cols = ['price', 'age', 'odometer',
              'manufacturer', 'condition', 'cylinders',
              'fuel', 'transmission', 'type', 'state']

df_model = df_clean[model_cols].copy()

df_model.head()

After cleaning, the dataset contains only observations with valid prices and non-missing values for the main features used in the analysis. The df_model table is the modeling-ready dataset that will be encoded and split into training and test sets.

In [ ]:
# 3.5 Encode categorical variables using one-hot encoding
categorical_cols = ['manufacturer', 'condition', 'cylinders',
                    'fuel', 'transmission', 'type', 'state']

df_encoded = pd.get_dummies(df_model, columns=categorical_cols,
                            drop_first=True)

df_encoded.head()

In [ ]:
# 3.6 Train-test split
X = df_encoded.drop('price', axis=1)
y = df_encoded['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape

## 4. Modeling

We start with a **Multiple Linear Regression** model as a baseline.  
Then we fit **Ridge** and **Lasso** regression with cross-validation and grid search over hyperparameters.

We will evaluate performance mainly with **Root Mean Squared Error (RMSE)**, which penalizes large errors and is commonly used for regression tasks where large mispricing can be costly.


### 4.1 Baseline: Multiple Linear Regression
This baseline provides a reference level of performance before applying regularization.

In [ ]:
# 4.1 Baseline: Multiple Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred_lr = lin_reg.predict(X_test)

mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

print(f"Linear Regression RMSE: {rmse_lr:,.2f}")
print(f"Linear Regression MAE : {mae_lr:,.2f}")

This baseline error serves as a benchmark for evaluating regularized models.


### 4.2 Cross-validation (baseline)

To evaluate how well the baseline linear regression model generalizes,
I compute the RMSE using 5-fold cross-validation on the training data.

In [ ]:
# 4.2 Cross-validation for the linear regression model
cv_scores = cross_val_score(
    lin_reg, X_train, y_train,
    scoring='neg_root_mean_squared_error',
    cv=5
)
rmse_cv = -cv_scores
print("Cross-validated RMSE (mean ± std): "
      f"{rmse_cv.mean():,.2f} ± {rmse_cv.std():.2f}")

The cross-validated mean RMSE and its standard deviation provide a more
robust estimate of expected performance than a single train/test split.

### 4.3 Ridge Regression (L2 regularization)

After one-hot encoding, the model includes many correlated features.
Ridge regression helps reduce overfitting by shrinking coefficients.
I tune the regularization strength (`alpha`) using grid search with 5-fold cross-validation

In [ ]:
# 4.3 Ridge Regression with Grid Search
ridge = Ridge()

ridge_params = {'alpha': [0.1, 1.0, 10.0, 100.0]}

ridge_grid = GridSearchCV(
    ridge,
    ridge_params,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)

ridge_grid.fit(X_train, y_train)

print("Best Ridge params:", ridge_grid.best_params_)
print("Best Ridge CV RMSE:", -ridge_grid.best_score_)

best_ridge = ridge_grid.best_estimator_
y_pred_ridge = best_ridge.predict(X_test)

mse_ridge = mean_squared_error(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mse_ridge)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)

print(f"Ridge Test RMSE: {rmse_ridge:,.2f}")
print(f"Ridge Test MAE : {mae_ridge:,.2f}")

Ridge typically improves stability when predictors are correlated.
Next, I fit Lasso regression to evaluate whether a sparse (feature-selecting) model performs better.


### 4.4 Lasso Regression (L1 regularization)

Lasso regression can shrink some coefficients all the way to zero, which may help with feature selection in a high-dimensional dataset.
I tune `alpha` using grid search with 5-fold cross-validation and compare performance to the baseline and Ridge.


In [ ]:
# 4.4 Lasso Regression with Grid Search
lasso = Lasso(max_iter=10000)

lasso_params = {'alpha': [0.001, 0.01, 0.1, 1.0]}

lasso_grid = GridSearchCV(
    lasso,
    lasso_params,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)

lasso_grid.fit(X_train, y_train)

print("Best Lasso params:", lasso_grid.best_params_)
print("Best Lasso CV RMSE:", -lasso_grid.best_score_)

best_lasso = lasso_grid.best_estimator_
y_pred_lasso = best_lasso.predict(X_test)

mse_lasso = mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mse_lasso)
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)

print(f"Lasso Test RMSE: {rmse_lasso:,.2f}")
print(f"Lasso Test MAE : {mae_lasso:,.2f}")

At this point, I have trained three models (Linear, Ridge, and Lasso).
In the next section, I compare RMSE and MAE across models and select the best-performing one for interpretation.


## 5. Evaluation

In this section, I compare the performance of the three regression models
(Linear Regression, Ridge Regression, and Lasso Regression) using **RMSE**
and **MAE** computed on the **test set**.

The model with the **lowest test RMSE** is selected as the best-performing
model and is used for coefficient interpretation.

In [ ]:
# Compare model performance on the test set
results = pd.DataFrame({
    'model': ['Linear Regression', 'Ridge Regression', 'Lasso Regression'],
    'RMSE': [rmse_lr, rmse_ridge, rmse_lasso],
    'MAE': [mae_lr, mae_ridge, mae_lasso]
})

results = results.sort_values(by='RMSE', ascending=True)
results

The table above summarizes predictive performance across models.
Lower RMSE indicates better accuracy, especially for large pricing errors.
Next, I programmatically select the best-performing model.

### 5.1 Selecting the best model

To avoid assumptions, I select the best-performing model based on the
lowest **test RMSE** observed in the evaluation results.

In [ ]:
rmse_dict = {
    "Linear Regression": rmse_lr,
    "Ridge Regression": rmse_ridge,
    "Lasso Regression": rmse_lasso
}

best_name = min(rmse_dict, key=rmse_dict.get)

if best_name == "Linear Regression":
    best_model = lin_reg
elif best_name == "Ridge Regression":
    best_model = best_ridge
else:
    best_model = best_lasso

print("Selected best model:", best_name)
print("Best test RMSE:", rmse_dict[best_name])

In [ ]:
With the best-performing model identified, I now analyze its coefficients
to understand which features are most strongly associated with used car prices.

### 5.2 Interpreting coefficients

Because categorical variables were one-hot encoded, each coefficient
represents the marginal effect of that feature relative to its reference
category, holding all other variables constant.

In [ ]:
coef_df = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': best_model.coef_
}).sort_values(by='coefficient', ascending=False)

coef_df.head(15)


In [ ]:
coef_df.tail(15)

Positive coefficients indicate features associated with higher prices,
while negative coefficients indicate features associated with lower prices.
These associations are descriptive and do not imply strict causality.


## 6. Business Interpretation & Recommendations

Based on the model:

- **Age** and **mileage** (odometer) typically have **negative coefficients**, indicating that older and higher-mileage cars sell for lower prices, all else equal.
- Certain **manufacturers** show positive coefficients, suggesting that these brands retain value better than others.
- Features such as **fuel type**, **transmission** (e.g., automatic), or **vehicle type** (e.g., SUV vs. sedan) may also contribute positively or negatively to price.

### Non-technical summary for used car dealers

1. **Prioritize newer, low-mileage vehicles.**  
   These vehicles command higher prices and are less likely to be discounted heavily.

2. **Focus on brands that retain value.**  
   Brands with consistently positive coefficients are good candidates for inventory expansion.

3. **Transmission and fuel type matter.**  
   In many markets, automatic transmission and hybrid/electric vehicles may attract higher willingness to pay.

4. **Use the model as a pricing assistant.**  
   The predicted price can be used as a reference when purchasing vehicles at auctions or setting retail prices.

### Next steps

- Explore non-linear models (e.g., tree-based ensembles) to capture more complex interactions.
- Segment the analysis by region or body type to refine pricing strategy.
- Deploy this model as part of a simple pricing dashboard for sales staff.
